In [ ]:
from scipy.integrate import odeint

In [ ]:
y = test.y
dy = differentiation_method(test.y)
y0 = [y[0], dy[0]]  # random inital state for x2
u = test.u
samples = y.shape[0]

In [ ]:
# initialize system's parameters
a1 = 10  #-0.764  # 0.443
a2 = 10  #-0.892  #-0.691
a3 = 10  # 0.797  # 1.407
b1 = 20  # 0.201  # 0.314

k = np.arange(0,samples,1)

def u_control(k):
    return u[int(k)]

def silverbox(x, k):
    x1 = x[0]
    x2 = x[1]
    dx1 = x2
    dx2 = -a1*x2 -a2*x1 -a3*x1**3+b1*u_control(k)
    return [dx1, dx2]

# y0 is our initial state
s = odeint(silverbox, y0=y0, t=k)

In [ ]:
plt.plot(y[:])
plt.plot(s[1:,0]-np.mean(s[1:,0]))
plt.show()
plt.plot(y[:])
plt.plot(s[:,1])
plt.show()